In [2]:
import numpy as np
import random
import csv
import copy
import random
import sys
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import pandas as pd
import math


class TspProblem():
    def __init__(self):
        self.cities = []
        self.population = []
        self.total_cost = []
        self.newPop = []
        self.givenSolution = []
        
    def distance(self,x, y): # Euclidean distance
        dist = np.linalg.norm(np.array(x)-np.array(y))
        return dist
    
    # Read file and generate cities list
    def readTSP(self,file): 
        with open(file, mode='r', newline='') as tsp:
            reader = csv.reader(tsp)
            for i in reader:
                self.cities.append(i)
    
    # Read Given solution list 
    def readSol(self,file): 
        with open(file, mode='r', newline='') as solution:
            reader = csv.reader(solution)
            for i in reader:
                self.givenSolution.append(int(i[0]))

    # Generate inital population with given solution
    def genPop(self,size): 
        for i in range(size):
            chromosome = self.givenSolution
            random.shuffle(chromosome)
            indexOfStart = chromosome.index(0) # find start point
            front = chromosome[indexOfStart:]
            back = chromosome[0:indexOfStart]
            chromosome = front + back
            chromosome.append(int(0))
            self.population.append(chromosome)  
        
    # calculate fitness using distance function
    def fitnessEval(self):
        fitnessEval = []
        for i in range(len(self.population)):
            distance = [] 
            total_cost = 0
            for idx in range(len(self.givenSolution)):
                pos_city_1 = [float(self.cities[self.population[i][idx]][0]), float(self.cities[self.population[i][idx]][1])]
                pos_city_2 = [float(self.cities[self.population[i][idx+1]][0]), float(self.cities[self.population[i][idx+1]][1])]
                distance.append(self.distance(pos_city_1, pos_city_2)) 
            
            total_cost = sum(distance)
            fitnessEval.append(total_cost) # 총 거리
        return fitnessEval
    
    # select parents using generated fitness evalutaion
    # two parents index will be returned
    # big wheel selection used
    # default weight is 2.0 and 0.5 ( K for good mod = 2.0 // bad mod = 0.5)
    def select(self,fitnessEval, weight_g = 2.0, weight_b= 0.5): 
        parents = []
        modFit = []
        for i in fitnessEval:
            modFit.append((len(self.givenSolution) * 100) - i)
        Idx = modFit.index(max(modFit))
        Idx2 = modFit.index(min(modFit))
        modFit[Idx] *= weight_g
        modFit[Idx2] *= weight_b
        max_value = sum(modFit)
        
        for j in range(2):
            pick = random.uniform(0, max_value)
            current = 0
            for i in modFit:
                current += i
                if current > pick:
                    parents.append(modFit.index(i))
                    break
        return parents
            
        
    # Crossover chromosome of selected parents using 
    def crossOver(self, selectIdx, mutateRate, mutateTime):
        child = [-1 for i in range(len(self.population[selectIdx[0]]))]
        ind = 0
        cross1 = (self.population[selectIdx[0]])
        cross2 = (self.population[selectIdx[1]])
        
        while True:
            # 자식의 값이 모두 채워지면 종료
            if child.count(-1)==0:
                break

            child[ind] = cross1[ind]
            ind = cross1.index(cross2[ind])
            
            if child[ind]!=-1 and child.count(-1)!=0:
                cross3 = cross1
                cross1 = cross2
                cross2 = cross3
                ind = child.index(-1)
        child.pop()
        child = self.mutate(child,mutateRate,mutateTime)
        child.append(child[0])
        
        return  self.newPop.append(child)  

    # Find near node Idx which selected at mutate fuction.
    # this function only used in self.mutate()
    # calculate distance and return near node idx
    def findCloseNode(self,chromosome,point):
        distance = [] # distance 초기화
        for idx in range(len(chromosome)):
            pos_city_1 = [float(self.cities[chromosome[point]][0]), float(self.cities[chromosome[point]][1])]
            pos_city_2 = [float(self.cities[chromosome[idx]][0]), float(self.cities[chromosome[idx]][1])]
            
            distance.append(self.distance(pos_city_1, pos_city_2)) # 두 도시의 거리를 스택, 총 1000개가 입력된다.
            if distance[-1] == 0: distance[-1] = 100000
        nearestNode = distance.index(min(distance)) 
        return nearestNode
    
    # mutate chromosome using findCloseNode function
    # should define MUTATION_RATE & MUTATION_TIME
    # random change mutation will be execute with low percentage
    def mutate(self,chromosome,mutateRate,mutateTime):
        if random.random() < mutateRate:
            for i in range(mutateTime):
                point1 = random.randint(1, len(chromosome)-2)
                nearestNode = self.findCloseNode(chromosome,point1)
                if chromosome[nearestNode] == 0: continue
                chromosome[point1+1], chromosome[nearestNode] = chromosome[nearestNode], chromosome[point1+1]
        if random.random()*100 < mutateRate:
            for i in range(mutateTime):
                print("이벤트 돌연변이")
                point1 = random.randint(1, len(chromosome)-2)
                point2 = random.randint(1, len(chromosome)-2)
                chromosome[point1], chromosome[point2] = chromosome[point2], chromosome[point1]
            
        return chromosome

    def changeGen(self):
        self.population = self.newPop.copy()
        self.newPop = []
    
    
class Cluster():
    def __init__(self,citiesAll):
        self.citiesAll = citiesAll
        self.centroidDistance = []
        self.centroids_of_cluster = []
        self.cities_of_cluster = []
        self.idxs_list = []
        self.cetarr = [] 
    
    def genDistOfCentroid(self):
        for i in range(len(self.centroids_of_cluster)):
            distance = []
            for idx in range(len(self.centroids_of_cluster)):
                pos_city_1 = [float(self.centroids_of_cluster[i][0]), float(self.centroids_of_cluster[i][1])]
                pos_city_2 = [float(self.centroids_of_cluster[idx][0]), float(self.centroids_of_cluster[idx][1])]
                
                distance.append(self.calculateEuclid(pos_city_1,pos_city_2))
            self.centroidDistance.append(distance)

    def calculateEuclid(self,x, y): # Euclidean distance
        dist = np.linalg.norm(np.array(x)-np.array(y))
        return dist

    def generateCentroid(self,clusterNumber=10):
        df = pd.DataFrame(self.citiesAll)
        df.columns = ['x', 'y']
        data = df
        k = clusterNumber
        model = KMeans(n_clusters = k, random_state = 10)
        model.fit(data)
        df['cluster'] = model.predict(data)
        centroids = model.cluster_centers_
        self.idxs_list = []
        self.centroids_of_cluster = centroids
        for i in range(0, k):
            tmp = [cities for cities in df[df['cluster'] == i].index]
            self.idxs_list.append(tmp)
            tmp_list = []
            for k in tmp :
                tmp_list.append(self.citiesAll[k])
            self.cities_of_cluster.append(tmp_list)

    def findCentroidIdx(self): 
        self.cetarr = []
        for p in range(10):
            self.cetarr.append(self.idxs_list[p][0])   
        X=-1  
        for q in range(10):
            if self.cetarr[q] == 0:
                X = q
                break


class Node():
    def __init__(self,heuristic,coordinate):
        self.coordinate = coordinate
        self.childNode = None
        self.heuristic = heuristic

#휴리스틱 함수
def heuri(currentNode, nodeList):
    tmpList = nodeList[:] # 임시 복사
    tmpList.remove(currentNode)
    minDistance = 99999
    for i in range (len(tmpList)):
        tmpdistance = getNodeCentroidDistance(currentNode,tmpList[i])
        if(minDistance > tmpdistance):
            minDistance = tmpdistance
    return minDistance

# 다음 경로를 선택
def chooseNextNode(nodeList, startNode,selectedNode):
    selectedNode.append(startNode) # 선택된 노드
    if len(nodeList) == 1: # 노드가 하나 남았으면 종료
        return None
    nodeList.remove(startNode)
    gPlusH = list()
    for i in range (len(nodeList)):
        g = getNodeCentroidDistance(startNode,nodeList[i])
        h = heuri(nodeList[i],nodeList)
        gPlusH.append(g+h)
    minGplusH = min(gPlusH) # g + h 중에서 가장 작은 노드를 선택
    minIndex = gPlusH.index(minGplusH)  # 새롭게 선택할 노드의 인덱스
    return nodeList[minIndex]
    
def getNodeCentroidDistance(node1, node2): 
            width = node1.coordinate[0] - node2.coordinate[0]
            height = node1.coordinate[1] - node2.coordinate[1]
            return math.sqrt(width*width+height*height)
    


  



In [7]:
 
if __name__ == '__main__':

    SIZEOFCHROMO = 1000
    INITPOPSIZE = 50
    mutateRate = 0.1
    mutateTime = 5
    POPSIZE = 10
    GENERATION = 10000


    f = open('stdout2.txt', 'w')

    a = TspProblem()
    a.readTSP('TSP.csv')
    a.readSol('example_solution.csv')
    a.genPop(INITPOPSIZE)                               

    for i in range(GENERATION):
        fitness = a.fitnessEval()
        for j in range(POPSIZE):
            parentsIdx = a.select(fitness)
            a.crossOver(parentsIdx,mutateRate,mutateTime)
        a.changeGen()
        minimum = min(fitness)    
        print("{0}, {1}".format(i,minimum))
        print("{0}, {1}".format(i,minimum), file = f)

    print("{0}세대, 최소 비용은 {1} ".format(i,minimum), file = f)
    fitIdx = fitness.index(min(fitness))
    print(a.population[fitIdx], file = f)
    f.close()

TypeError: crossOver() takes 2 positional arguments but 4 were given